# Network Deliberation ABM
## Discrete state, generated NK Model task

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches': None}

# Imports
import configparser
import math

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import netdelib.soclearn as slearn
import numpy as np
import pandas as pd
import repsci

# Configure plotting in Jupyter
from matplotlib import pyplot as plt

mpl.rcParams['font.size'] = 7
mpl.rcParams['figure.dpi'] = 600
mpl.rcParams['lines.markersize'] = 7

symbols = ['.','.','.','.']
colors = ['#848','#88f','#f88', '#b6b']
plt.figure(figsize=(7.5,10))
exclude = ['Random', 'StochasticBlock', 'Lattice']
reproduce = [
    '2021-01-06 130248 bc84b67'
]

In [ ]:
def load_result_list(experiments):
    strategies = []
    networks = []
    
    score = {}
    correct = {}
    distance = {}
    initial_distance = {}
    converge_time = {}
    
    for experiment_id in experiments:
        experiment, exp_score, exp_correct, exp_distance, exp_initial, exp_converge = load_results(experiment_id)
        
        exp_strategies = experiment.config.get('strategies', 'enabled').split(',')
        for i, s in enumerate(exp_strategies):
            
            if s not in strategies:
                strategies.append(s)
                score[s] = {}
                correct[s] = {}
                distance[s] = {}
                initial_distance[s] = {}
                converge_time[s] = {}
            
            exp_networks = experiment.config.get('networks', 'enabled').split(',')
            for j, n in enumerate(exp_networks):
                
                if n not in networks:
                    networks.append(n)
                
                score[s][n] = exp_score[i][j]
                correct[s][n] = exp_correct[i][j]
                distance[s][n] = exp_distance[i][j]
                if exp_initial is not None:
                    initial_distance[s][n] = exp_initial[i][j]
                if exp_converge is not None:
                    converge_time[s][n] = exp_converge[i][j]
    
    return experiment, strategies, networks, score, correct, distance, initial_distance, converge_time
    
def load_result_run_list(experiments):
    strategies = []
    networks = []
    
    score = {}
    correct = {}
    distance = {}
    initial_distance = {}
    converge_time = {}
    
    for experiment_id in experiments:
        experiment, exp_score, exp_correct, exp_distance, exp_initial, exp_converge = load_result_runs(experiment_id)
        
        exp_strategies = experiment.config.get('strategies', 'enabled').split(',')
        for i, s in enumerate(exp_strategies):
            
            if s not in strategies:
                strategies.append(s)
                score[s] = {}
                correct[s] = {}
                distance[s] = {}
                initial_distance[s] = {}
                converge_time[s] = {}
            
            exp_networks = experiment.config.get('networks', 'enabled').split(',')
            for j, n in enumerate(exp_networks):
                
                if n not in networks:
                    networks.append(n)
                
                score[s][n] = exp_score[i][j]
                correct[s][n] = exp_correct[i][j]
                distance[s][n] = exp_distance[i][j]
                if exp_initial is not None:
                    initial_distance[s][n] = exp_initial[i][j]
                if exp_converge is not None:
                    converge_time[s][n] = exp_converge[i][j]
    
    return experiment, strategies, networks, score, correct, distance, initial_distance, converge_time
    
def load_results (experiment_id):
    
    experiment = repsci.Experiment(
        "discrete_generated_nk",
        reproduce=experiment_id)
    config = experiment.get_config()
    
    score = np.load(experiment.get_filename('score.npy'))
    correct = np.load(experiment.get_filename('correct.npy'))
    distance_score = np.load(experiment.get_filename('distance_score.npy'))
    # Older code did not produce the below data
    try:
        initial_distance = np.load(experiment.get_filename('initial_distance.npy'))
    except FileNotFoundError:
        initial_distance = None
    try:
        converge_time = np.load(experiment.get_filename('converge_time.npy'))
    except FileNotFoundError:
        converge_time = None
    return experiment, score, correct, distance_score, initial_distance, converge_time

def load_result_runs (experiment_id):
    
    experiment = repsci.Experiment(
        "discrete_generated_nk",
        reproduce=experiment_id)
    config = experiment.get_config()
    
    run_score = np.load(experiment.get_filename('score_run.npy'))
    run_correct = np.load(experiment.get_filename('correct_run.npy'))
    run_distance_score = np.load(experiment.get_filename('distance_score_run.npy'))
    # Older code did not produce the below data
    try:
        run_initial_distance = np.load(experiment.get_filename('initial_distance_run.npy'))
    except FileNotFoundError:
        run_initial_distance = None
    try:
        run_converge_time = np.load(experiment.get_filename('converge_time_run.npy'))
    except FileNotFoundError:
        run_converge_time = None
    return experiment, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time

def individual_title(config):
    individual_mode = config.getint('strategies', 'individual_mode', fallback=1)
    if config.getboolean('strategies', 'individual', fallback=False):
#        if config.getboolean('strategies', 'individual_all_bits', fallback=True):
#            title_individual = 'Hill-Climb'
#        else:
#            title_individual = 'Mutation'
        title_individual = ''
        if individual_mode == slearn.MODE_ALL:
            title_individual += 'Serial-Indiv.'
        elif individual_mode == slearn.MODE_FALLBACK:
            title_individual += 'Fallback-Indiv.'
        elif individual_mode == slearn.MODE_BEST:
            title_individual += 'Parallel-Indiv.'
    else:
        title_individual = 'social-only'
    return title_individual

def result_title (config):
    individual_mode = config.getint('strategies', 'individual_mode', fallback=1)
    if config.getboolean('strategies', 'individual', fallback=False):
        if config.getboolean('strategies', 'individual_all_bits', fallback=True):
            title_individual = 'hill-climb'
        else:
            title_individual = 'mutation'
        if individual_mode == slearn.MODE_ALL:
            title_individual += ' serial indiv.'
        elif individual_mode == slearn.MODE_FALLBACK:
            title_individual += ' fallback indiv.'
        elif individual_mode == slearn.MODE_BEST:
            title_individual += ' parallel indiv.'
    else:
        title_individual = 'social-only'
    title = "|V|={} N={}, K={}, EXP={}, S={}, {}, {}".format(
        config.getint('abm', 'N'),
        config.getint('abm', 'bit_count'),
        config.getint('abm', 'K'),
        config.getfloat('abm', 'nk_exponent', fallback=1),
        config.getint('strategies', 'sample', fallback=None),
        {True: 'critical', False: 'non-critical'}[config.getboolean('strategies', 'critical')],
        title_individual)
    return title

def nested_label(data, fig, ax, width, height=1, offset=(0, 0), ):

    label_trans = mpl.transforms.blended_transform_factory(
        fig.transFigure, ax.transAxes)

    layer_width = width[0]
    cell_height = height / len(data)
    
    for i, data in enumerate(reversed(data)):
        rect = mpl.patches.Rectangle(
            (offset[0] + lw, offset[1] + i * cell_height), width=layer_width, height=cell_height,
            lw=2, edgecolor='black', facecolor="none",
            transform=label_trans, clip_on=False)
#        plt.text(
#            offset[0] + 0.02, offset[1] + i * cell_height + 0.04, data['label'],
#            fontsize=18,
#            transform=label_trans, rotation='vertical')
        plt.text(
            offset[0] + 0.02, offset[1] + (i + 0.5) * cell_height, data['label'],
            fontsize=18, verticalalignment='center',
            transform=label_trans, rotation='vertical')
        ax.add_patch(rect)
        if 'data' in data:
            inner_offset = (
                offset[0] + layer_width,
                offset[1] + (i) * cell_height)
            nested_label(data['data'], fig, ax, width[1:], height=cell_height, offset=inner_offset)


In [ ]:
fig = plt.figure(figsize=(3, 6))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

smallworld = [max(rs) for rs in run_score['Confident Neighbor']['Small World']]
randompod = [max(rs) for rs in run_score['Confident Neighbor']['Random Group']]

plt.subplot(4, 1, 1)
plt.hist(
    [smallworld, randompod],
    bins=np.arange(0,1.025,0.025), label=['Small-World', 'ND Random-Pod'],
    zorder=2)
plt.grid()
plt.xlabel('Solution Quality (Confident-Neighbor)')
plt.ylabel('count')
plt.ylim([0, 200])
plt.xlim([-0.025, 1.025])
plt.legend(fontsize=5, loc="upper left")
x = -0.205
plt.text(x, 210, 'a)')

plt.subplot(4,1,2)

smallworld = [max(rs) for rs in run_score['Conform']['Small World']]
randompod = [max(rs) for rs in run_score['Conform']['Random Group']]

plt.hist(
    [smallworld, randompod],
    bins=np.arange(0,1.025,0.025), label=['Small-World', 'ND Random-Pod'],
    zorder=2)
plt.grid()
plt.xlabel('Solution Quality (Conform)')
plt.ylabel('count')
plt.ylim([0, 200])
plt.xlim([-0.025, 1.025])
plt.legend(fontsize=5, loc="upper left")
x = -0.205
plt.text(x, 210, 'b)')

plt.subplot(4,1,3)
diff_conform = [
    max(run_score['Conform']['Random Group'][r]) - max(run_score['Conform']['Long Path'][r]) for r in range(1000)
]
diff_bestneighbor = [
    max(run_score['Best Neighbor']['Random Group'][r]) - max(run_score['Best Neighbor']['Long Path'][r]) for r in range(1000)
]
plt.hist(
    [diff_bestneighbor, diff_conform],
    bins=np.arange(-1.025,1.075,.05),
    label=["Best-Neighbor", "Conform"],
    zorder=2)
plt.grid()
plt.xlabel('ND-Random-Pod - ND-Long-Path')
plt.ylabel('count')
plt.ylim([0,430])
plt.xlim([-0.8, 0.8])
plt.legend(fontsize=5, loc="upper left")
x = -0.57 * 1.6 / (0.97 - 0.12)
plt.text(x, 440, 'c)')

plt.subplot(4,1,4)
diff_pa = [
    max(run_score['Confident Neighbor']['Pref. Attach.'][r]) - max(run_score['Best Neighbor']['Pref. Attach.'][r]) for r in range(1000)
]
diff_sw = [
    max(run_score['Confident Neighbor']['Small World'][r]) - max(run_score['Best Neighbor']['Small World'][r]) for r in range(1000)
]
plt.hist(
    [diff_pa, diff_sw],
    bins=np.arange(-1.025,1.075,.05),
    label=["Pref.-Attach.", "Small-World"],
    zorder=2)
plt.grid()
plt.xlabel('Confident-Neighbor - Best-Neighbor')
plt.ylim([0,500])
plt.ylabel('count')
plt.xlim([-0.8, 0.8])
plt.legend(fontsize=5, loc="upper left")

plt.tight_layout()
plt.subplots_adjust(top=0.97, left=0.15, right=0.97)
x = -0.57 * 1.6 / (0.97 - 0.12)
plt.text(x, 510, 'd)')

plt.savefig(experiment.get_filename('fig-results-netdelib-diff-all.png'), dpi=600)
plt.savefig(experiment.get_filename('fig-results-netdelib-diff-all.eps'))

In [ ]:
fig = plt.figure(figsize=(6, 3))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

plt.subplot(2,2,1)
plt.hist(
    [max(rs) for rs in run_score['Conform']['Random Group']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Conform: Random Group')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,200])

plt.subplot(2,2,3)
plt.hist(
    [max(rs) for rs in run_score['Conform']['Long Path']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Conform: Long-Path')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,200])

plt.subplot(2,2,2)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Random Group']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Best-Neighbor: Random Group')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,200])

plt.subplot(2,2,4)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Long Path']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Best-Neighbor: Long Path')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,200])

plt.tight_layout()
plt.savefig(experiment.get_filename('results-netdelib-efficiency-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-netdelib-efficiency-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(6, 3))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

plt.subplot(2,2,1)
plt.hist(
    [max(rs) for rs in run_score['Confident Neighbor']['Pref. Attach.']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Conf-Nbr: Pref-Attach')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,250])
plt.yticks([0, 100, 200])

plt.subplot(2,2,3)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Pref. Attach.']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Best-Nbr: Pref-Attach')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,250])
plt.yticks([0, 100, 200])

plt.subplot(2,2,2)
plt.hist(
    [max(rs) for rs in run_score['Confident Neighbor']['Small World']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Conf-Nbr: Small-World')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,250])
plt.yticks([0, 100, 200])

plt.subplot(2,2,4)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Small World']],
    bins=np.arange(0,1.025,.025), zorder=2)
plt.grid()
plt.title('Best-Nbr: Small-World')
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.ylim([0,250])
plt.yticks([0, 100, 200])

plt.tight_layout()
plt.savefig(experiment.get_filename('results-confident-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-confident-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 3))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

diff = [
    max(run_score['Confident Neighbor']['Pref. Attach.'][r]) - max(run_score['Best Neighbor']['Pref. Attach.'][r]) for r in range(1000)
]
plt.subplot(2,1,1)
plt.hist(diff, bins=np.arange(-1.025,1.075,.05), label="Pref-Attach", zorder=2)
plt.grid()
plt.title('Difference in Solution Quality')
plt.ylabel('Count')
plt.ylim([0,500])
plt.legend(fontsize=6, loc='upper left')
locs, labels = plt.xticks()
plt.xticks(locs, labels=[])
plt.xlim([-1.075, 1.075])

diff = [
    max(run_score['Confident Neighbor']['Small World'][r]) - max(run_score['Best Neighbor']['Small World'][r]) for r in range(1000)
]
plt.subplot(2,1,2)
plt.hist(diff, bins=np.arange(-1.025,1.075,.05), label="Small-World", zorder=2)
plt.grid()
plt.ylabel('Count')
plt.xlabel('Confident-Neighbor - Best-Neighbor')
plt.ylim([0,500])
plt.xlim([-1.075, 1.075])
plt.legend(fontsize=6, loc='upper left')

plt.tight_layout()
plt.savefig(experiment.get_filename('results-confident.png'), dpi=600)
plt.savefig(experiment.get_filename('results-confident.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-08 133817 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

ylim = 250
pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Best-Neighbor (Parallel)')
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Pref. Attach.']],
    label="Pref-Attach", zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")
plt.ylabel('Count')
plt.yticks([0, 100, 200])

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Small World']],
    label="Small-World", zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")
plt.ylabel('Count')
plt.yticks([0, 100, 200])

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Long Path']],
    label="Long-Path", zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")
plt.ylabel('Count')
plt.yticks([0, 100, 200])

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.yticks([0, 100, 200])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-bn-parallel-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-bn-parallel-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-08 133817 86fe505'
]

strat = 'Confident Neighbor'


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Confident-Neighbor (Parallel)')
plt.hist(
    [max(rs) for rs in run_score[strat]['Pref. Attach.']],
    label='Pref-Attach', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score[strat]['Small World']],
    label='Small-World', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score[strat]['Long Path']],
    label='Long-Path', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score[strat]['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-cn-parallel-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-cn-parallel-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-08 133817 86fe505'
]

strat = 'Conform'


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Conform (Parallel)')
plt.hist(
    [max(rs) for rs in run_score[strat]['Pref. Attach.']],
    label='Pref-Attach', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score[strat]['Small World']],
    label='Small-World', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score[strat]['Long Path']],
    label='Long-Path', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score[strat]['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-conf-parallel-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-conf-parallel-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-04 161927 86fe505'
]


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Best-Neighbor (Fallback)')
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Pref. Attach.']],
    label='Pref-Attach', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Small World']],
    label='Small-World', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Long Path']],
    label='Long-Path', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score['Best Neighbor']['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-bn-fallback-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-bn-fallback-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-04 161927 86fe505'
]

strat = 'Confident Neighbor'


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Confident-Neighbor (Fallback)')
plt.hist(
    [max(rs) for rs in run_score[strat]['Pref. Attach.']],
    label='Pref-Attach', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score[strat]['Small World']],
    label='Small-World', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score[strat]['Long Path']],
    label='Long-Path', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score[strat]['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")


plt.tight_layout()
plt.savefig(experiment.get_filename('results-cn-fallback-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-cn-fallback-dist.eps'))

In [ ]:
fig = plt.figure(figsize=(3, 4.5))

reproduce = [
    '2021-02-04 161927 86fe505'
]

strat = 'Conform'


experiment, learning_strategies, networks, run_score, run_correct, run_distance_score, run_initial_distance, run_converge_time = load_result_run_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

pos = np.arange(0,1.2,0.2)

plt.subplot(4,1,1)
plt.title('Conform (Fallback)')
plt.hist(
    [max(rs) for rs in run_score[strat]['Pref. Attach.']],
    label='Pref-Attach', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.ylim([0,ylim])
plt.xticks(pos, labels=[])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,2)
plt.hist(
    [max(rs) for rs in run_score[strat]['Small World']],
    label='Small-World', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,3)
plt.hist(
    [max(rs) for rs in run_score[strat]['Long Path']],
    label='Long-Path', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xticks(pos, labels=[])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.subplot(4,1,4)
plt.hist(
    [max(rs) for rs in run_score[strat]['Random Group']],
    label='Random-Pod', zorder=2,
    bins=np.arange(0,1.025,.025))
plt.grid()
plt.ylabel('Count')
plt.yticks([0, 100, 200])
plt.xlabel('Solution Quality')
plt.xticks(pos, [0, 0.2, 0.4, 0.6, 0.8, 1])
plt.ylim([0,ylim])
plt.legend(fontsize=6, loc="upper left")

plt.tight_layout()
plt.savefig(experiment.get_filename('results-conf-fallback-dist.png'), dpi=600)
plt.savefig(experiment.get_filename('results-conf-fallback-dist.eps'))